Loading the Required Libraries and Modules

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
plt.style.use('fivethirtyeight')

import re
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package stopwords to /home/chi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/chi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/chi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


 Loading the Data and Performing Basic Data Checks

In [12]:
text = pd.read_csv('user_reviews_g1.csv')
text.head()

,id,review,type_review,date,language,views,votes,score
0,lutri,Maybe the best game I have ever played. It jus...,normal,2021-03-22,English,0,0,9
1,lolnolol,Not worth your time or your money. Supposedly ...,expanded,2021-03-22,English,0,0,0
2,Svetlendrius,A good shooter-horror adventure with the belov...,normal,2021-03-21,English,0,0,8
3,Hodgeftm,"This game has awoken my love for gaming, I wen...",expanded,2021-03-20,English,0,0,10
4,DaPonti,"Very well executed and not too repetitive, unl...",expanded,2021-03-20,English,0,0,9


Data Cleaning

In [13]:
text['review'] = text['review'].str.lower()
text.head()

,id,review,type_review,date,language,views,votes,score
0,lutri,maybe the best game i have ever played. it jus...,normal,2021-03-22,English,0,0,9
1,lolnolol,not worth your time or your money. supposedly ...,expanded,2021-03-22,English,0,0,0
2,Svetlendrius,a good shooter-horror adventure with the belov...,normal,2021-03-21,English,0,0,8
3,Hodgeftm,"this game has awoken my love for gaming, i wen...",expanded,2021-03-20,English,0,0,10
4,DaPonti,"very well executed and not too repetitive, unl...",expanded,2021-03-20,English,0,0,9


In [14]:
import re
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from collections import Counter


punctuations = string.punctuation
stopwords = stopwords.words('english')
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /home/chi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
def cleanup_text1(review):
    review = review.lower()
    review = re.sub('\[.*?\]', '', review)
    review = re.sub('https?://\S+|www\.\S+', '', review)
    review = re.sub('<.*?>+', '', review)
    review = re.sub('[%s]' % re.escape(string.punctuation), '', review)
    review = re.sub('\n', '', review)
    review = re.sub('\w*\d\w*', '', review)
    return review

Creating the Training and Test Datasets

In [16]:
X = text['review']
target = text['type_review']

Converting Text to Word Frequency Vectors with CountVectorizer 

In [17]:
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(text['review'].values.astype('U'))

Spliting the training and test datasets

In [18]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.30, random_state = 100)

print(text.shape)
print(X_train.shape)
print(X_test.shape)

(2719, 8)
(1903, 14374)
(816, 14374)


Create and Fit the Classifier.

In [19]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()

In [20]:
nb.fit(X_train, y_train)

MultinomialNB()

In [21]:
prediction=nb.predict(X_test)
print(prediction[: 10])

['normal' 'expanded' 'normal' 'expanded' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal']


Create a confusion matrix and classification report

In [22]:
from sklearn.metrics import confusion_matrix, classification_report

In [23]:
print(confusion_matrix(y_test, prediction))

[[393 207]
 [ 44 172]]


In [24]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

    expanded       0.90      0.66      0.76       600
      normal       0.45      0.80      0.58       216

    accuracy                           0.69       816
   macro avg       0.68      0.73      0.67       816
weighted avg       0.78      0.69      0.71       816



Using XGBoost algorithm to optimize the result

In [25]:
import xgboost as xgb
from xgboost import XGBClassifier

In [26]:
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [27]:
y_pred = classifier.predict(X_test)

In [28]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [29]:
cm

array([[578,  22],
       [ 49, 167]])

Building Random Forest Classifier

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [31]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 100)

classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=100)

In [32]:
predRF = classifier.predict(X_test) 
print(predRF[:10])

# Calculate the accuracy score
accuracy_RF = metrics.accuracy_score(y_test, predRF)
print(accuracy_RF)

Conf_metrics_RF = metrics.confusion_matrix(y_test, predRF, labels=['normal', 'expanded'])
print(Conf_metrics_RF)



['expanded' 'expanded' 'normal' 'expanded' 'normal' 'expanded' 'normal'
 'expanded' 'expanded' 'normal']
0.8725490196078431
[[116 100]
 [  4 596]]
